# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model
from azureml.automl.core.featurization import FeaturizationConfig
from azureml.interpret import ExplanationClient
from azureml.pipeline.steps import AutoMLStep


In [2]:
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


### Workspace 

In [3]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="automl-udacity-capstone")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-135197
Azure region: southcentralus
Subscription id: 976ee174-3882-4721-b90a-b5fef6b72f24
Resource group: aml-quickstarts-135197


### Compute Cluster

In [4]:
from azureml.core import Workspace
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

ws = Workspace.from_config() # this automatically looks for a directory .azureml

#Create compute cluster
cluster_name="automl-project"

try:
  compute_target=ComputeTarget(workspace=ws,name=cluster_name)
  print("Found existing compute target")
except ComputeTargetException:
  print('Creating a new compute target')
  compute_config=AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                       max_nodes=5, 
                                                       idle_seconds_before_scaledown=2400,
                                                       vm_priority='lowpriority')
  compute_target=ComputeTarget.create(ws,cluster_name,compute_config)
  

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes =  2)

Creating a new compute target
Creating
Succeeded.....................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [5]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

Research-project ComputeInstance Succeeded
automl-project AmlCompute Succeeded


In [6]:
# For a more detailed view of current cluster status, use the 'status' property
print(compute_target.status.serialize())

{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-01-18T11:12:25.239000+00:00', 'errors': None, 'creationTime': '2021-01-18T11:12:18.356962+00:00', 'modifiedTime': '2021-01-18T11:12:34.333365+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 5, 'nodeIdleTimeBeforeScaleDown': 'PT2400S'}, 'vmPriority': 'LowPriority', 'vmSize': 'STANDARD_D2_V2'}


## Dataset

### Overview

**Citation Source** :

Creator:

B. German
Central Research Establishment
Home Office Forensic Science Service
Aldermaston, Reading, Berkshire RG7 4PN

Donor:

Vina Spiehler, Ph.D., DABFT
Diagnostic Products Corporation
(213) 776-0180 (ext 3014)


**Data Set Information**:

Vina conducted a comparison test of her rule-based system, BEAGLE, the nearest-neighbor algorithm, and discriminant analysis. BEAGLE is a product available through VRS Consulting, Inc.; 4676 Admiralty Way, Suite 206; Marina Del Ray, CA 90292 (213) 827-7890 and FAX: -3189. In determining whether the glass was a type of "float" glass or not, the following results were obtained (# incorrect answers):

Type of Sample -- Beagle -- NN -- DA
Windows that were float processed (87) -- 10 -- 12 -- 21
Windows that were not: (76) -- 19 -- 16 -- 22

The study of classification of types of glass was motivated by criminological investigation. At the scene of the crime, the glass left can be used as evidence...if it is correctly identified!


Attribute Information:

1. Id number: 1 to 214
2. RI: refractive index
3. Na: Sodium (unit measurement: weight percent in corresponding oxide, as are attributes 4-10)
4. Mg: Magnesium
5. Al: Aluminum
6. Si: Silicon
7. K: Potassium
8. Ca: Calcium
9. Ba: Barium
10. Fe: Iron
11. Type of glass: (class attribute)
- 1 building_windows_float_processed
- 2 building_windows_non_float_processed
- 3 vehicle_windows_float_processed
- 4 vehicle_windows_non_float_processed (none in this database)
- 5 containers
- 6 tableware
- 7 headlamps


Relevant Papers:

Ian W. Evett and Ernest J. Spiehler. Rule Induction in Forensic Science. Central Research Establishment. Home Office Forensic Science Service. Aldermaston, Reading, Berkshire RG7 4PN
[Web Link]


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [7]:
# Import dataset
from azureml.core import Dataset
from azureml.data.dataset_factory import DataType

# create a TabularDataset from a delimited file behind a public web url 
web_path ="https://raw.githubusercontent.com/Aishwaryasasanapuri/Cervical-cancer-prediction/main/kag_risk_factors_cervical_cancer.csv"
final_df = Dataset.Tabular.from_delimited_files(path=web_path) 

# preview the first 3 rows of ds
final_df.take(3).to_pandas_dataframe()

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
2,34,1.0,?,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [8]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_hours" : 0.6,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 10,
    "max_concurrent_iterations": 8,
    "max_cores_per_iteration": -1,
    "n_cross_validations": 5,
    "primary_metric": 'AUC_weighted',
    "featurization": 'auto',
    "verbosity": logging.INFO,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             compute_target=compute_target,
                             training_data = final_df,
                             label_column_name = 'Biopsy',
                             iterations= 100,
                             **automl_settings
                            )

In [9]:
# TODO: Submit your experiment
automl_run = exp.submit(automl_config)

Running on remote.


In [10]:
automl_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-udacity-capstone,AutoML_71d2287b-a6b7-4166-874e-c61c9a93f791,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
from azureml.widgets import RunDetails
RunDetails(run).show()

# Wait for the remote run to complete
automl_run.wait_for_completion(show_output=True)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number

{'runId': 'AutoML_71d2287b-a6b7-4166-874e-c61c9a93f791',
 'target': 'automl-project',
 'status': 'Completed',
 'startTimeUtc': '2021-01-18T11:16:13.347575Z',
 'endTimeUtc': '2021-01-18T11:42:55.66916Z',
 'properties': {'num_iterations': '100',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'automl-project',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-udacity-capstone","subscription_id":"976ee174-3882-4721-b90a-b5fef6b72f24","resource_group":"aml-quickstarts-135197","workspace_name":"quick-starts-ws-135197","region":"southcentralus","compute_target":"automl-project","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":100,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":nu

In [12]:
automl_run.get_metrics('accuracy')

{'accuracy': 0.940561675506596}

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.


Retrieve the Best Model's explanation from the best_run which includes explanations for engineered features and raw features. Make sure that the run for generating explanations for the best model is completed.

In [13]:
# Wait for the best model explanation run to complete
from azureml.core.run import Run
model_explainability_run_id = automl_run.id + "_" + "ModelExplain"
print(model_explainability_run_id)
model_explainability_run = Run(experiment=exp, run_id=model_explainability_run_id)
model_explainability_run.wait_for_completion()

# Get the best run object
automl_best_run, model = automl_run.get_output()
automl_best_run_metrics = automl_best_run.get_metrics()

AutoML_71d2287b-a6b7-4166-874e-c61c9a93f791_ModelExplain


In [14]:
automl_best_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-udacity-capstone,AutoML_71d2287b-a6b7-4166-874e-c61c9a93f791_57,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [15]:
print(model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                reg_alpha=0.9473684210526315,
                                                                                                reg_lambda=0,
                                                                                                silent=True,
                                               

In [16]:
print('Best Run Id: ', automl_best_run.id)
print('\n Accuracy:', automl_best_run_metrics['accuracy'])
print('\n AUC Weighted:', automl_best_run_metrics['AUC_weighted'])

Best Run Id:  AutoML_71d2287b-a6b7-4166-874e-c61c9a93f791_57

 Accuracy: 0.940561675506596

 AUC Weighted: 0.9670063876922667


#### Describing the Best model

In [17]:
from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['31', '36', '45', '6', '7', '35', '28', '15', '26'],
 'weights': [0.06666666666666667,
             0.13333333333333333,
             0.06666666666666667,
             0.06666666666666667,
             0.06666666666666667,
             0.2,
             0.13333333333333333,
             0.2,
             0.06666666666666667]}

31 - sparsenormalizer
{'copy': True, 'norm': 'l2'}

31 - gradientboostingclassifier
{'ccp_alpha': 0.0,
 'criterion': 'mse',
 'init': None,
 'learning_rate': 0.046415888336127774,
 'loss': 'deviance',
 'max_depth': 10,
 'max_features': 0.6,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,


In [18]:
model_name = automl_best_run.properties['model_name']
model_name

'AutoML71d2287ba57'

In [19]:
#TODO: Save the best model
import joblib

os.makedirs('./outputs', exist_ok=True)
#best_model_automl=

joblib.dump(model,filename='outputs/best_automlmodel.pkl')

['outputs/best_automlmodel.pkl']

In [20]:
automl_best_run.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":0,"CurrentNodeCount":5}',
 'ensembled_iterations': '[31, 36, 45, 6, 7, 35, 28, 15, 26]',
 'ensembled_algorithms': "['GradientBoosting', 'XGBoostClassifier', 'GradientBoosting', 'ExtremeRandomTrees', 'ExtremeRandomTrees', 'ExtremeRandomTrees', 'RandomForest', 'ExtremeRandomTrees', 'LightGBM']",
 'ensemble_weights': '[0.06666666666666667, 0.13333333333333333, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.2, 0.13333333333333333, 0.2, 0.06666666666666667]',
 'best_individual_pipeline_score': '0.9585241429561044',
 'best_individual_iteration': '31',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True',
 'ModelExplainRunId': '87bf4384-4e4a-454b-b1df-23d4886b6aff',
 'model_explain_run_id': '87bf4384-4e4a-454b-b1df-23d4886b6aff',
 'model_explanation': 'True'}

## Registering Model

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [21]:
from azureml.core.model import Model

model = Model.register(workspace = ws,
                        model_path ='outputs/best_automlmodel.pkl',
                        model_name = "automlmodel")

Registering model automlmodel


In [22]:
print(model)
print(model.id)

Model(workspace=Workspace.create(name='quick-starts-ws-135197', subscription_id='976ee174-3882-4721-b90a-b5fef6b72f24', resource_group='aml-quickstarts-135197'), name=automlmodel, id=automlmodel:1, version=1, tags={}, properties={})
automlmodel:1
